# LEXICON SCORING ALGORITHM - 2/7/24

## Importing all necessary libraries

In [1]:
# Import all the necessary Libraries
import pandas as pd

## Reading the data

In [9]:
# Reading the data 
df = pd.read_csv("C:/Users/LENOVO/OneDrive/Desktop/TextBasedAnalysis/deceptive-opinion.csv")
df.reset_index(drop=True, inplace=True)
df.head(10)

,deceptive,hotel,polarity,source,text
0,truthful,conrad,positive,TripAdvisor,We stayed for a one night getaway with family ...
1,truthful,hyatt,positive,TripAdvisor,Triple A rate with upgrade to view room was le...
2,truthful,hyatt,positive,TripAdvisor,This comes a little late as I'm finally catchi...
3,truthful,omni,positive,TripAdvisor,The Omni Chicago really delivers on all fronts...
4,truthful,hyatt,positive,TripAdvisor,I asked for a high floor away from the elevato...
5,truthful,omni,positive,TripAdvisor,I stayed at the Omni for one night following a...
6,truthful,conrad,positive,TripAdvisor,We stayed in the Conrad for 4 nights just befo...
7,truthful,omni,positive,TripAdvisor,Just got back from 2 days up in Chicago shoppi...
8,truthful,omni,positive,TripAdvisor,We arrived at the Omni on 2nd September for a ...
9,truthful,hyatt,positive,TripAdvisor,"On our visit to Chicago, we chose the Hyatt du..."


## Exploring Opinion Lexicon in NLTK Library

In [10]:
# Import necessary Libraries
from sklearn import preprocessing
import nltk
nltk.download('opinion_lexicon')
# Importing the opinion_lexicon corpus, which contains positive&negative opinion words
from nltk.corpus import opinion_lexicon
# Importing word_tokenize function, which is used for tokenizing words
from nltk.tokenize import word_tokenize

print('Total number of words in opinion lexicon', len(opinion_lexicon.words()))
print('Examples of positive words in opinion lexicon',
      opinion_lexicon.positive()[:10])
print('Examples of negative words in opinion lexicon',
      opinion_lexicon.negative()[:10])

Total number of words in opinion lexicon 6789
Examples of positive words in opinion lexicon ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']
Examples of negative words in opinion lexicon ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']


[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


## Creation of Dictionary for Sentiment Analysis

In [11]:
# Let's create a dictionary which we can use for scoring our review text
nltk.download('punkt')
df.rename(columns={"reviewText": "text"}, inplace=True)
pos_score = 1
neg_score = -1
word_dict = {}
 
# Adding the positive words to the dictionary
for word in opinion_lexicon.positive():
        word_dict[word] = pos_score
      
# Adding the negative words to the dictionary
for word in opinion_lexicon.negative():
        word_dict[word] = neg_score


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Calculating Sentiment Score with Bing Liu Lexicon

In [12]:
#Creating a fuunction text
def bing_liu_score(text):
    #Initializing the sentiment score
    sentiment_score = 0
    #Tokenizing the input text into words and convert them to lowercase
    bag_of_words = word_tokenize(text.lower())
     # creating loop to check each word in the bag of words
    for word in bag_of_words:
        #Checking if the word exists in the sentiment dictionary
        if word in word_dict:
            # If the word exist, adding its sentiment score to the sentiment score
            sentiment_score += word_dict[word]
    return sentiment_score  #Returning the sentiment score for the text

In [14]:
# Fill NaN values in the 'text' column
df['text'].fillna('NA', inplace=True)
#creating new column 'Bing_Liu_Score' to store the scores by applying  bing_liu_score to calculate sentiment scores for each text
df['Bing_Liu_Score'] = df['text'].apply(bing_liu_score)

In [ ]:
# Displaying the first 10 rows of the DataFrame with few columns
df[["text", 'Bing_Liu_Score']].head(10)

## Calculating mean sentiment score

In [15]:
# Grouping 'overall' rating and calculating the mean sentiment score using the Bing Liu Lexicon
df.groupby('text').agg({'Bing_Liu_Score':'mean'})

,Bing_Liu_Score
text,
"I stayed at the Sheraton Chicago Hotel and Towers a couple nights ago. The service was excellent!! As soon as i stepped through the door they grabbed my bags, and were helping me choose the best hotel room for me! I loved there service and the room was gorgeous! I highly recommend it for anyone, and there prices aren't as bad as others. I loved the experience and I'm sure next time I'm in the area I'll visit again!\n",6.0
"If you are traveling anywhere near Lake Michigan within the next year or less, I would highly recommend staying at The Sofitel Hotel. Unbeatable elegance, fine dining, and magnificent views are only just a few amenities to speak of. Along with excellent service and reasonable pricing you will be assured to get a quality nights rest. If there were such a thing as a 10 star rating, The Sofitel would surely have it. My visit and stay there was nothing short of elegance and comfort. Located minutes from everything in downtown Chicago, one would hardly want to leave. Fine dining, dancing, and an evening bar will leave you absolutely relaxing after any meeting, or social event in town. The Sofitel Hotel will leave you wanting to visit the moment you leave.\n",10.0
"If you are traveling to Chicago and need a place to stay, I highly recommend the Hotel Allegro Chicago. It was visiting time visiting Chicago for the first time on a business trip and a friend referred me to the Hotel Allegro. He said they were pet friendly,which was a huge plus. This is because I hate leaving my kitten muffin with a pet sitter every time I travel for business. Upon my arrival, I was met with friendly valet who treated me like I was the only customer who mattered. I walked in and saw a hotel like no other I have seen before or after. The lobby was decorated with modern furniture and paintings and looked like a place I could relax in. After I had entered and checked in, I saw the staircase, which looked like a testament to the staircase in the 'Titanic'. Frankly, I was amazed and I had not even entered my room yet. My room was a Queen Deluxe room and it was amazing. The bed had a blue headboard,which I have never seen before, and the colors worked together so well I seriously contemplated living there. The room service was exceptional and promptly brought the food I ordered. Later I wanted to get a drink and I enjoyed it at The Bar 312, which is conveniently located adjacent to the hotel. These are the specifics of my stay, but there is much more to be experienced that I did not mention if you visit the Hotel Allegro. In summary, they allow pets, the service is amazing, the decoration superb, and it is conveniently situated next to dining establishments. What more could a person ask for?\n",18.0
"My wife and I's stay at the Sheraton Chicago Hotel and Towers proved to be the perfect complement to our time spent visiting and exploring the Windy City. Upon our arrival, we were immediately welcomed by the beautiful design of the first-floor lobby and the friendly demeanor of the desk and bell service staff. In seemingly no time at all we were in our clean and well-furnished room with a beautiful and peaceful view of Lake Michigan. In terms of accommodations, I have never personally stayed at a hotel that I could complement more. On our very first night, my wife took advantage of a message therapy session while I swam a few laps in the heated indoor swimming pool. We followed our time apart with a very relaxing time at the Chi bar, which had a very enlivening and laid-back atmosphere. Retiring to our rooms, I couldn't help but remark to my wife about the quality of our room and the bed in particular. I had a very comfortable night's sleep and woke up just on time thanks to a spot-on wake up call from the desk staff. After a quick bite to eat the LB Bistro, we headed out to sight-see. Overall, I can't recommend the Sheraton Chicago Hotel and Towers enough. As far as this customer is concerned, it is the only place to sta